# Acidentes de tráfico de bicicletas

Vamos a utilizar un dataset de accidentes de tráfico de bicicletas que provienen del portal de datos abiertos de Madrid

https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=20f4a87ebb65b510VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default


Una vez preprocesados los datos tendremos un dataset con los siguientes campos:


- Número de expediente del accidente
- Fecha del accidente
- Distrito donde ha ocurrido el accidente
- Tipo de accidente
- Sexo de la persona que ha tenido el accidente
    

Vamos a intentar resolver las siguientes preguntas:

- Datos del número de expediente 2020S000044
- Accidentes que ocurren en el distrito de Moratalaz ordenados por fecha (Descendente)
- Número de accidentes por tipo de accidente (sin ningún orden establecido)
- Número de accidentes cuyo tipo es Colisión frontal

Preprocesado de datos

Se realizará un preprocesado de datos cuyo objetico es limpiar, simplificar y normalizar los datos para que la prática resulte más sencilla

El formato de salida será un fichero Microsoft Excel que contendrá los datos limpios.



## Carga de datos

In [ ]:
import pandas as pd

Leemos los datos ...

In [ ]:
df = pd.read_csv("../../data/AccidentesBicicletas_2020.txt", delimiter=";")

## Exploración y limpieza

Limpiamos los datos ...

In [ ]:
df.info()

In [ ]:
df = df.assign(Fecha = pd.to_datetime(df.Fecha, format="%d/%m/%Y"),
               Min = pd.to_numeric(df.Hora.str.split(":").str.get(1)),
               Hora = pd.to_numeric(df.Hora.str.split(":").str.get(0))
              )
df = df.filter(["NumExpediente", "Fecha", "Distrito", "TipoAccidente", "Sexo"])
# Ponemos las columnas en minúsculas para no se inserten en Postgres con Dobles Comillas
df = df.rename(columns = {"NumExpediente" : "num_expediente",
                           "Fecha" : "fecha",
                           "Distrito": "distrito",
                           "TipoAccidente" : "tipo_accidente",
                           "Sexo": "sexo"}
              )
df.info()

In [ ]:
df.sample(10)

Eliminamos los expedientes duplicados para simplificar el ejemplo ...

In [ ]:
ExpedientesDuplicados = pd.DataFrame(df.groupby("num_expediente").size()).reset_index().rename( columns = {0 : 'n'}).query("n > 1").num_expediente
df = df[~df.num_expediente.isin(ExpedientesDuplicados)]
df = df[df.sexo.notnull()]
df = df.reset_index(drop = True)
df.info()

In [ ]:
df.sample(10)

## Grabación de los datos limpios

Y finalmente escribimos el resultado a un fichero Excel ,,,

In [ ]:
writer = pd.ExcelWriter("../../data/accidentes.xlsx")
df.to_excel(writer, 'Data', index=False)
writer.save()